# Create a Marquee Portfolio with GS Quant

The Marquee Portfolio Service provides a powerful framework for uploading portfolio positions and retrieving analytics including historical performance, factor risk exposure, ESG analytics, and more. GS Quant makes operating the suite of Portfolio Service API's intuitive and fast.

## Permission Prerequisites

To execute all the code in this tutorial, you will need the following application scopes:
- **read_product_data**
- **read_financial_data**
- **modify_financial_data** (must be requested)
- **run_analytics** (must be requested)
- **read_user_profile**

If you are not yet permissioned for these scopes, please request them on your [My Applications Page](https://developer.gs.com/go/apps/view). If you have any other questions please reach out to the [Marquee sales team](mailto:gs-marquee-sales@gs.com).


## Step 1: Authenticate and Initialize Your Session

First you will import the necessary modules and add your client id and client secret.

In [ ]:
import itertools

from gs_quant.api.gs.assets import GsAssetApi
from gs_quant.api.gs.portfolios import GsPortfolioApi
from gs_quant.common import PositionSet
from gs_quant.entities.entitlements import Entitlements, EntitlementBlock, User
from gs_quant.markets.portfolio_manager import PortfolioManager
from gs_quant.session import GsSession, Environment
from gs_quant.target.portfolios import Portfolio as MQPortfolio
from gs_quant.target.portfolios import Position

client = 'ENTER CLIENT ID'
secret = 'ENTER CLIENT SECRET'


GsSession.use(Environment.PROD, client_id=client, client_secret=secret, scopes=('read_product_data read_financial_data modify_financial_data run_analytics read_user_profile',))

print('GS Session initialized.')

## Step 2: Define Portfolio Entitlements

By default, an application will have all entitlement permissions to a portfolio it makes. However, if you would like to share the portfolio with others, either Marquee users or other applications, you will need to specify them in the entitlements parameter of the portfolio. Let's walk through how we convert a list of admin and viewer emails into an `Entitlements` object:

In [ ]:
portfolio_admin_emails = ['LIST OF ADMIN EMAILS']
portfolio_viewer_emails = ['LIST OF VIEWER EMAILS']

admin_entitlements = EntitlementBlock(users=User.get_many(emails=portfolio_admin_emails))
view_entitlements = EntitlementBlock(users=User.get_many(emails=portfolio_viewer_emails))

entitlements = Entitlements(view=view_entitlements,
                            admin=admin_entitlements)

print(f'Entitlements:\n{entitlements.to_dict()}')

## Step 3: Define the Positions You Would Like to Upload

Portfolio positions in Marquee are stored on a holding basis, when means you only upload positions for days where you are rebalancing your portfolio. Take the following set of positions:

In [ ]:
positions = [
    { 'identifier': 'GS UN', 'quantity': 50, 'positionDate': '2020-05-01'},
    { 'identifier': 'AAPL UW', 'quantity': 25, 'positionDate': '2020-05-01'},
    { 'identifier': 'GS UN', 'quantity': 51, 'positionDate': '2020-07-01'},
    { 'identifier': 'AAPL UW', 'quantity': 26, 'positionDate': '2020-07-01'}
]

If these positions were to be uploaded correctly, this portfolio would hold 50 shares of GS UN and 25 shares of AAPL UW from May 1, 2020 to June 30, 2020, and it would hold 51 shares of GS UN and 26 shares of AAPL UW from July 1, 2020 to today.

## Step 4: Format positions

Now let's proceed with updating these positions to our portfolio. The first step is to resolve the identifiers provided into their corresponding unique Marquee identifiers. In this case, positions are identified by Bloomberg ID, but other identifiers can be used and resolved by adding them to the `fields` parameter in the function `GsAssetApi.resolve_assets`.

In [ ]:
all_identifiers = list(set([p['identifier'] for p in positions]))
results = GsAssetApi.resolve_assets(identifier=all_identifiers, fields=['bbid', 'id'], limit=1)
try:
    identifier_to_marquee_id = dict(zip(results.keys(), [a[0]['id'] for a in results.values()]))
except:
    unmapped_assets = {k for k,v in results.items() if not v}
    raise ValueError('Error in resolving the following identifiers: ' + ', '.join(unmapped_assets))

print('Position identifiers successfully mapped as the following:')
for mq_id in identifier_to_marquee_id:
    print(f'{mq_id} --> {identifier_to_marquee_id[mq_id]}')

Next we need to rearrange the data in the positions to fit the format expected in the Marquee API.

In [ ]:
portfolio_position_sets = []

for position_date, positions_on_date in itertools.groupby(positions, lambda x: x['positionDate']):
    formatted_positions = tuple(Position(asset_id=identifier_to_marquee_id[p['identifier']],
                                    quantity=p['quantity']) for p in positions_on_date)
    position_set = (PositionSet(position_date=position_date,
                                positions=formatted_positions))
    portfolio_position_sets.append(position_set)

print('Portfolio positions successfully formatted.')

## Step 5: Create the Portfolio
We're finally ready to create our portfolio and update it with our newly formatted positions.

In [ ]:
# Format and create a new empty portfolio
portfolio = GsPortfolioApi.create_portfolio(MQPortfolio(name='ENTER PORTFOLIO NAME HERE',
                                                      currency='USD',
                                                      entitlements=entitlements.to_target()))
print(f"Created portfolio '{portfolio.name}' with ID: {portfolio.id}")

GsPortfolioApi.update_positions(portfolio.id, portfolio_position_sets)
print(f"Updated positions in '{portfolio.name}'")


## Step 6: Schedule Reports
By default, creating a portfolio will automatically create a corresponding Performance Report for it as well. If you would like to create a Factor Risk Report for it as well, follow the steps [here](../examples/marquee/00_create_factor_risk_report.ipynb). Then, remember to schedule all the portfolio reports with our `PortfolioManager` class.
a

In [ ]:
pm = PortfolioManager(portfolio.id)
pm.schedule_reports()

print('All portfolio reports scheduled.')

### You're all set, Congrats! What's next?

* [Creating and scheduling a factor risk report](../examples/marquee/00_create_factor_risk_report.ipynb)

* [Updating the portfolio with new positions](../tutorials/Update%20Historical%20Portfolio.ipynb)

* [Retrieving the portfolio's performance analytics](../tutorials/Pull%20Portfolio%20Performance%20Data.ipynb)

* [Retrieving the portfolio's factor risk and attribution analytics](../tutorials/Pull%20Portfolio%20Factor%20Risk%20Data.ipynb)


*Other questions? Reach out to the [Portfolio Analytics team](mailto:gs-marquee-analytics-support@gs.com)!*